# <u>Teams</u>

AgentChat creates **agent teams** for them to interact with each other.

Each team achieves **a goal** by working together.

**First we will**
1. Create a team
2. Run a team

While doing this we will **observe team's behavior**, which is crucial along with common **operations to control team's behavior**

**Team presets**
1. RoundRobinGroupChat - runs group chat with agents participating sequentially (covered here)
2. SelectorGroupChat - (advanced) team that selects the next participant using a judge (LLM)
3. MagneticOneGroupChat - (advanced) generalist multi-agent to solve open ended web and file-based tasks
4. Swarm - (advanced)

**When to use a team?**

First start with single agent. Use all it's resources and if it still fails to solve the problem, use a team.

## <u>RoundRobinGroupChat</u>

### <u>Creating a team</u>

All agents share the same context.

All agents take turn to answer sequuentially.

Each agent broadcasts its answer to all other agents.

Example below : 2 Agents, *reflection pattern*, TextMentionTermination.

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini"
)

vendor_agent = AssistantAgent(
    "vendor",
    model_client=model_client,
    system_message="You are a vegetable vendor from Pune, India. Once some vegetables are sold, reply with 'THANK YOU'",
)

buyer_agent = AssistantAgent(
    "buyer",
    model_client=model_client,
    system_message="You are vegetable buyer conversing with a vendor in subji mandi. Ask for the vegetables you want and buy them at appropriate price.",
)

text_termination = TextMentionTermination("THANK YOU")

team = RoundRobinGroupChat([vendor_agent, buyer_agent], termination_condition=text_termination)

### RUn

In [5]:
result = await team.run(task="Write a short poem about the fall season.")
print(result)

messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 15, 17, 18, 56, 905831, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='vendor', models_usage=RequestUsage(prompt_tokens=43, completion_tokens=77), metadata={}, created_at=datetime.datetime(2025, 9, 15, 17, 19, 7, 544153, tzinfo=datetime.timezone.utc), content="In the crisp air, leaves gently sway,  \nGolden hues dance as they decay,  \nPumpkins gleam while the harvest calls,  \nNature's canvas, painted in stalls.  \n\nWarm spices waft from kitchens near,  \nCider and warmth, as winter draws near,  \nA tapestry woven with colors so bright,  \nFall wraps us in beauty, a cozy delight.  ", type='TextMessage'), TextMessage(source='buyer', models_usage=RequestUsage(prompt_tokens=131, completion_tokens=64), metadata={}, created_at=datetime.datetime(2025, 9, 15, 17, 19, 10, 124166, tzinfo=datetime.timezone.utc),

In [8]:
for i in result.messages:
    print(i.source, "\n", "----------", "\n", i.content, "\n")

user 
 ---------- 
 Write a short poem about the fall season. 

vendor 
 ---------- 
 In the crisp air, leaves gently sway,  
Golden hues dance as they decay,  
Pumpkins gleam while the harvest calls,  
Nature's canvas, painted in stalls.  

Warm spices waft from kitchens near,  
Cider and warmth, as winter draws near,  
A tapestry woven with colors so bright,  
Fall wraps us in beauty, a cozy delight.   

buyer 
 ---------- 
 That’s a lovely poem! Now, shifting back to the subji mandi, I’d like to buy some vegetables. Can you tell me the prices for the following?

- Potatoes
- Spinach
- Cauliflower
- Carrots
- Onions

Please let me know your best price for each! 

vendor 
 ---------- 
 Sure! Here are the best prices for the vegetables you requested:

- Potatoes: ₹20 per kg  
- Spinach: ₹30 per bunch  
- Cauliflower: ₹40 each  
- Carrots: ₹25 per kg  
- Onions: ₹15 per kg  

Let me know how much you would like to buy! 

buyer 
 ---------- 
 Great, thank you for the prices! I would like